In [ ]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd

import sys
sys.path.append('../solver')

from solver import Instance

### Matplotlib and seaborn settings

In [ ]:
markers = ['o', '*', 'v', '^', '<', 's']
sns.set_palette(palette=['#003f5c','#444e86','#955196','#dd5182','#ff6e54', '#ffa600'])
plt.rcParams.update({"text.usetex": True, "font.family" : "serif", "font.serif" : ["Computer Modern Serif"]})

### Load city info

In [ ]:
city_pop_k = {
    'Paris': 2244,
    'Lyon': 882,
    'Berlin': 975,
    'Frankfurt': 582
}

city_info = dict()

for instance in glob('../instances/*.json'):
    name = path.splitext(path.basename(instance))[0]
    city_info[name] = dict()
    
    with open(instance) as f:
        i = json.load(f)

    surface, pop, n_areas = 0, 0, 0

    for region in i['geography']['city']['regions']:
        surface += sum(area['surface_area'] for area in region['areas'])
        pop += sum(area['population'] for area in region['areas'])
        n_areas += len(region['areas'])

    avg_area = surface / n_areas

    if any(c in name for c in ('paris', 'lyon')):
        avg_area /= 1e6
    else:
        avg_area *= 1e4

    city_info[name]['avg_area_surface'] = avg_area
    city_info[name]['avg_area_pop'] = pop / n_areas
    city_info[name]['n_areas'] = n_areas
    city_info[name]['avg_area_pct'] = 100 / n_areas

    n_parcels = 0

    for scenario in i['scenarios']:
        for area_data in scenario['data']:
            n_parcels += sum(area_data['demand'])

    city_info[name]['n_parcels'] = n_parcels / i['num_scenarios']

### Load results

In [ ]:
keep_keys = [
    "instance",
    "model",
    "city",
    "DB",
    "DT",
    "OC",
    "RM",
    "GM",
    "num_periods",
    "num_scenarios",
    "obj_value",
    "elapsed_time",
    "n_variables",
    "n_constraints",
    "n_nonzeroes",
    "hiring_costs",
    "outsourcing_costs",
    "regional_avg_hired_pct",
    "global_avg_hired_pct"
]

optional_keys = [
    "n_shift_start_periods",
    "periods_with_start",
    "periods_with_start_pct",
    "courier_moved_pct"
]

models = ('MBase', 'Flex', 'PartFlex', 'Fixed')
models_replace = dict(base='MBase', flex='Flex', partflex='PartFlex', fixed='Fixed')
models_long = ('MBase', 'Flex', 'PartFlex ($\\mu = 4$)', 'PartFlex ($\\mu = 3$)', 'PartFlex ($\\mu = 2$)', 'Fixed')
models_textsc = {
    'MBase': '\\textsc{MBase}',
    'Flex': '\\textsc{Flex}',
    'PartFlex ($\\mu = 4$)': '\\textsc{PartFlex} ($\\mu = 4$)',
    'PartFlex ($\\mu = 3$)': '\\textsc{PartFlex} ($\\mu = 3$)',
    'PartFlex ($\\mu = 2$)': '\\textsc{PartFlex} ($\\mu = 2$)',
    'Fixed': '\\textsc{Fixed}'
}
demand_types_replace = dict(uniform='Uniform', at_end='AtEnd', double_peak='DoublePeak', peak='Peak')

def get_results(csv_path):
    if not path.exists(csv_path):
        raise FileNotFoundError(f"Results file not found: {csv_path}")
    
    d = pd.read_csv(csv_path)
    
    d.DT = pd.Categorical(d.DT, categories=demand_types_replace.values(), ordered=True)

    # For old files:
    d.model.replace({'BaseModel': 'MBase'}, inplace=True)
    d.model_long.replace({'BaseModel': 'MBase'}, inplace=True)
    
    d.model_long = pd.Categorical(d.model_long, categories=models_long, ordered=True)

    return d

In [ ]:
d = get_results(csv_path='../results/complete.csv')
du = get_results(csv_path='../results/uncapacitated.csv')

### Plotting functions

In [ ]:
def add_fig_legend(fig, bbox_to_anchor=(.5,-0.095)):
    elements = [
        (
            Patch(facecolor='C0', edgecolor='C0'),
            Line2D([0], [0], color='C0', marker=markers[0], markersize=8)
        ),
        (
            Patch(facecolor='C1', edgecolor='C1'),
            Line2D([0], [0], color='C1', marker=markers[1], markersize=8)
        ),
        (
            Patch(facecolor='C2', edgecolor='C2'),
            Line2D([0], [0], color='C2', marker=markers[2], markersize=8)
        ),
        (
            Patch(facecolor='C3', edgecolor='C3'),
            Line2D([0], [0], color='C3', marker=markers[3], markersize=8)
        ),
        (
            Patch(facecolor='C4', edgecolor='C4'),
            Line2D([0], [0], color='C4', marker=markers[4], markersize=8)
        ),
        (
            Patch(facecolor='C5', edgecolor='C5'),
            Line2D([0], [0], color='C5', marker=markers[5], markersize=8)
        ),
        Patch(facecolor='white', edgecolor='black'),
        Patch(facecolor='white', edgecolor='black', hatch='\\\\')
    ]
    
    fig.legend(
        handles=elements, handler_map={tuple: HandlerTuple(ndivide=2)},
        labels=[
            '\\textsc{MBase}', '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)', '\\textsc{PartFlex} ($\\mu = 2$)',
            '\\textsc{Fixed}', 'Outsourcing Costs', 'Hiring costs'
        ],
        fontsize=14, loc='lower center', bbox_to_anchor=bbox_to_anchor,
        ncols=4, handlelength=5, frameon=False
    )

    return fig

### Cost overview

* Boxplot of model vs. cost per parcel
* Barplot of model vs. cost per parcel of stacked{outsourced, inhouse} parcels

In [ ]:
def plot_cost_overview(d):
    plt.rcParams['hatch.linewidth'] = 2

    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,6))

    ax = axes.flat[0]

    sns.boxplot(data=d, x='model_long', y='cost_per_parcel', order=models_long, hue='model_long', hue_order=models_long, fliersize=0, ax=ax)
    ax.set_ylim((0, d.cost_per_parcel.quantile(0.999)))
    ax.set_ylabel('Avg cost per parcel', fontsize=16)
    ax.set_xlabel('')

    for idx, model in enumerate(models_long):
        avg = d[d.model_long == model].cost_per_parcel.median()
        ax.text(x=idx, y=avg, s=f"{avg:.2f}", va='bottom', ha='center', color='white', fontsize=16, fontweight='bold')

    ax = axes.flat[1]

    ticks = range(len(models_long))
    ax.bar(
        x=ticks,
        height=d.groupby('model_long', observed=True).outsourcing_costs_per_parcel.mean(),
        color=[f"C{i}" for i in ticks],
        linewidth=2,
        edgecolor='white')

    first_patches = ax.patches[:]

    ax.bar(
        x=ticks,
        height=d.groupby('model_long', observed=True).hiring_costs_per_parcel.mean(),
        bottom=d.groupby('model_long', observed=True).outsourcing_costs_per_parcel.mean(),
        color=[f"C{i}" for i in ticks],
        linewidth=2,
        edgecolor='white'
    )

    for bar in ax.patches:
        if bar in first_patches:
            continue
        bar.set_hatch('\\')

    ax.set_ylim((0, 1))
    ax.set_xticks(ticks)
    ax.set_xticklabels(models_long)
    ax.yaxis.tick_right()
    ax.set_ylabel('Avg cost per parcel', rotation=270, labelpad=20, fontsize=16)
    ax.yaxis.set_label_position('right')
    ax.yaxis.grid(which='major')
    ax.axes.set_axisbelow(True)
    ax.set_xlabel('')

    ax.legend(handles=[
        Patch(facecolor='white', edgecolor='black', label='Outsourcing costs'),
        Patch(facecolor='white', edgecolor='black', hatch='\\\\', label='Hiring costs')
    ], loc='upper center', ncols=2, frameon=False, fontsize=16)

    for ax in axes.flat:
        ticklabels = [lbl.get_text() for lbl in ax.get_xticklabels()]
        ticklabels = [models_textsc[lbl] for lbl in ticklabels]
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ticklabels, fontsize=16)

        for tl in ax.get_xticklabels():
            tl.set_rotation_mode('anchor')
            tl.set_rotation(45)
            tl.set_ha('right')
        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    fig.tight_layout()
    fig.savefig('cost_overview.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_cost_overview(d=d);

### Impact of the regional upper bound

In [ ]:
def plot_rm_impact(d):
    plt.rcParams['hatch.linewidth'] = 1

    fig, ax = plt.subplots(figsize=(10,6))

    n_models = len(d.model_long.unique())
    n_rm = len(d.RM.unique())

    ticks = [(x1 + x2) / 2 for x1, x2 in zip(range(1, n_rm * n_models, n_models + 1), range(n_models, n_rm * (n_models + 1), n_models + 1))]

    for rm_idx, rm in enumerate(d.RM.sort_values().unique()):
        for model_idx, model_long in enumerate(d.model_long.sort_values().unique()):
            bar_x = rm_idx * (n_models + 1) + model_idx + 1

            data = d[(d.RM == rm) & (d.model_long == model_long)]
            hc = data.hiring_costs_per_parcel.mean()
            oc = data.outsourcing_costs_per_parcel.mean()

            ax.bar(x=[bar_x], height=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white')
            ax.bar(x=[bar_x], height=[hc], bottom=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white', hatch='\\')

    ax.set_xticks(ticks)
    ax.set_xticklabels(d.RM.unique())
    ax.set_xlabel('\\texttt{RM}', fontsize=16, labelpad=10)
    ax.set_ylabel('Avg cost per parcel', fontsize=16)
    ax.yaxis.grid(which='major')
    ax.set_axisbelow(True)

    for tl in ax.get_xticklabels():
        tl.set_fontsize(14)
    for tl in ax.get_yticklabels():
        tl.set_fontsize(14)

    ax.tick_params(which='major', axis='x', length=0)

    ax.legend(handles=[
            Patch(facecolor='C0', edgecolor='C0'),
            Patch(facecolor='C1', edgecolor='C1'),
            Patch(facecolor='C2', edgecolor='C2'),
            Patch(facecolor='C3', edgecolor='C3'),
            Patch(facecolor='C4', edgecolor='C4'),
            Patch(facecolor='C5', edgecolor='C5'),
            Patch(facecolor='white', edgecolor='black'),
            Patch(facecolor='white', edgecolor='black', hatch='\\\\')
        ],
        labels=[
            '\\textsc{BaseModel}', '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)', '\\textsc{PartFlex} ($\\mu = 2$)',
            '\\textsc{Fixed}', 'Outsourcing Costs', 'Hiring costs'
        ],
        loc='lower center', bbox_to_anchor=(0.5, -0.4),
        ncol=4, fontsize=14, frameon=False,
        handletextpad=0.25
    )

    fig.tight_layout()
    fig.savefig('rm_vs_cost.pdf', dpi=96, bbox_inches='tight')

    return fig, ax

In [ ]:
plot_rm_impact(d=d);

### Impact of the demand baseline

In [ ]:
def plot_db_impact(d):
    plt.rcParams['hatch.linewidth'] = 1
    
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,6))

    ax = axes.flat[0]

    sns.lineplot(
        data=d, x='DB', y='parcels_outsourced_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.legend().set_visible(False)
    ax.axes.set_axisbelow(True)
    ax.axes.set_xticks(d.DB.unique())
    ax.set_xlabel('\\texttt{DB}', fontsize=16)
    ax.set_ylabel('Parcels outsourced (\\%)', fontsize=16)

    for db in d.DB.unique():
        ax.axvline(x=db, linestyle='--', color='black', linewidth=0.5, alpha=0.25)

    ax = axes.flat[1]

    n_models = len(d.model_long.unique())
    n_db = len(d.DB.unique())

    ticks = [(x1 + x2) / 2 for x1, x2 in zip(range(1, n_db* n_models, n_models + 1), range(n_models, n_db * (n_models + 1), n_models + 1))]

    for db_idx, db in enumerate(d.DB.sort_values().unique()):
        for model_idx, model_long in enumerate(d.model_long.sort_values().unique()):
            bar_x = db_idx * (n_models + 1) + model_idx + 1

            data = d[(d.DB == db) & (d.model_long == model_long)]
            hc = data.hiring_costs_per_parcel.mean()
            oc = data.outsourcing_costs_per_parcel.mean()

            ax.bar(x=[bar_x], height=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white')
            ax.bar(x=[bar_x], height=[hc], bottom=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white', hatch='\\')

    ax.set_xticks(ticks)
    ax.set_xticklabels(d.DB.unique())
    ax.set_xlabel('\\texttt{DB}', fontsize=16)
    ax.yaxis.tick_right()
    ax.set_ylabel('Avg cost per parcel', rotation=270, labelpad=20, fontsize=16)
    ax.yaxis.set_label_position('right')
    ax.yaxis.grid(which='major')
    ax.set_axisbelow(True)
    ax.tick_params(which='major', axis='x', length=0)

    for ax in axes.flat:
        for tl in ax.get_xticklabels():
            tl.set_fontsize(14)
        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    add_fig_legend(fig, bbox_to_anchor=(0.5, -0.15))

    fig.tight_layout()
    fig.savefig('db.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_db_impact(d);

### Impact of the outsourcing costs

In [ ]:
def plot_oc_impact(d):
    plt.rcParams['hatch.linewidth'] = 1
    
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,6))

    ax = axes.flat[0]

    sns.lineplot(
        data=d, x='OC', y='parcels_outsourced_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.legend().set_visible(False)
    ax.axes.set_axisbelow(True)
    ax.axes.set_xticks(d.OC.unique())
    ax.set_xlabel('\\texttt{OC}', fontsize=16)
    ax.set_ylabel('Parcels outsourced (\\%)', fontsize=16)

    for db in d.OC.unique():
        ax.axvline(x=db, linestyle='--', color='black', linewidth=0.5, alpha=0.25)

    ax = axes.flat[1]

    n_models = len(d.model_long.unique())
    n_oc = len(d.OC.unique())

    ticks = [(x1 + x2) / 2 for x1, x2 in zip(range(1, n_oc * n_models, n_models + 1), range(n_models, n_oc * (n_models + 1), n_models + 1))]

    for oc_idx, oc in enumerate(d.OC.sort_values().unique()):
        for model_idx, model_long in enumerate(d.model_long.sort_values().unique()):
            bar_x = oc_idx * (n_models + 1) + model_idx + 1

            data = d[(d.OC == oc) & (d.model_long == model_long)]
            hcm = data.hiring_costs_per_parcel.mean()
            ocm = data.outsourcing_costs_per_parcel.mean()

            ax.bar(x=[bar_x], height=[ocm], color=f"C{model_idx}", linewidth=1, edgecolor='white')
            ax.bar(x=[bar_x], height=[hcm], bottom=[ocm], color=f"C{model_idx}", linewidth=1, edgecolor='white', hatch='\\')

    ax.set_xticks(ticks)
    ax.set_xticklabels(d.OC.unique())
    ax.set_xlabel('\\texttt{OC}', fontsize=16)
    ax.yaxis.tick_right()
    ax.set_ylabel('Avg cost per parcel', rotation=270, labelpad=20, fontsize=16)
    ax.yaxis.set_label_position('right')
    ax.yaxis.grid(which='major')
    ax.set_axisbelow(True)
    ax.tick_params(which='major', axis='x', length=0)

    for ax in axes.flat:
        for tl in ax.get_xticklabels():
            tl.set_fontsize(14)
        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    add_fig_legend(fig, bbox_to_anchor=(0.5, -0.15))

    fig.tight_layout()
    fig.savefig('oc.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_oc_impact(d);

### Impact of the demand type

In [ ]:
def plot_dt_impact(d):
    plt.rcParams['hatch.linewidth'] = 1
    
    fig, ax = plt.subplots(figsize=(12,6))

    n_models = len(d.model_long.unique())
    n_dt = len(d.DT.unique())

    ticks = [(x1 + x2) / 2 for x1, x2 in zip(range(1, n_dt * n_models, n_models + 1), range(n_models, n_dt * (n_models + 1), n_models + 1))]

    for dt_idx, dt in enumerate(d.DT.sort_values().unique()):
        for model_idx, model_long in enumerate(d.model_long.sort_values().unique()):
            bar_x = dt_idx * (n_models + 1) + model_idx + 1

            data = d[(d.DT == dt) & (d.model_long == model_long)]
            hc = data.hiring_costs_per_parcel.mean()
            oc = data.outsourcing_costs_per_parcel.mean()

            ax.bar(x=[bar_x], height=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white')
            ax.bar(x=[bar_x], height=[hc], bottom=[oc], color=f"C{model_idx}", linewidth=1, edgecolor='white', hatch='\\')1

    ax.set_xticks(ticks)
    ax.set_xticklabels(d.DT.sort_values().unique())
    ax.set_xlabel('\\texttt{DT}', fontsize=16)
    ax.set_ylabel('Avg cost per parcel', fontsize=16)
    ax.yaxis.grid(which='major')
    ax.set_axisbelow(True)

    ticklabels = [lbl.get_text() for lbl in ax.get_xticklabels()]
    ticklabels = [
        f"\\textsc{{{txt}}}" for txt in ticklabels
    ]
    ax.set_xticklabels(ticklabels, fontsize=16)

    for tl in ax.get_yticklabels():
        tl.set_fontsize(14)

    ax.tick_params(which='major', axis='x', length=0)

    ax.legend(handles=[
            Patch(facecolor='C0', edgecolor='C0'),
            Patch(facecolor='C1', edgecolor='C1'),
            Patch(facecolor='C2', edgecolor='C2'),
            Patch(facecolor='C3', edgecolor='C3'),
            Patch(facecolor='C4', edgecolor='C4'),
            Patch(facecolor='C5', edgecolor='C5'),
            Patch(facecolor='white', edgecolor='black'),
            Patch(facecolor='white', edgecolor='black', hatch='\\\\')
        ],
        labels=[
            '\\textsc{BaseModel}', '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)', '\\textsc{PartFlex} ($\\mu = 2$)',
            '\\textsc{Fixed}', 'Outsourcing Costs', 'Hiring costs'
        ],
        loc='lower center', bbox_to_anchor=(0.5, -0.4),
        ncol=4, fontsize=16, frameon=False,
        handletextpad=0.25
    )

    fig.tight_layout()
    fig.savefig('dt.pdf', dpi=96, bbox_inches='tight')

    return fig, ax

In [ ]:
plot_dt_impact(d);

In [ ]:
def plot_dt_ops(d):
    fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(18,6))
    
    ax = axes.flat[0]

    sns.lineplot(
        data=d, x='DT', y='parcels_outsourced_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_ylabel('Parcels outsourced (\\%)', fontsize=16)

    ax = axes.flat[1]

    sns.lineplot(
        data=d, x='DT', y='global_avg_hired_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_ylabel('Couriers hired as a \\% of the global limit', fontsize=16)
    
    ax = axes.flat[2]

    sns.lineplot(
        data=d, x='DT', y='courier_moved_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_ylabel('Couriers moving between areas (\\%)', fontsize=16)

    for ax in axes.flat:
        for dt in range(len(d.DT.unique())):
            ax.axvline(x=dt, linestyle='--', color='black', linewidth=0.5, alpha=0.25)
        
        ax.legend().set_visible(False)
        ax.axes.set_axisbelow(True)
        ax.set_xlabel('\\texttt{DT}', fontsize=16, labelpad=10)
        ax.set_xticks(ax.get_xticks())

        ticklabels = [lbl.get_text() for lbl in ax.get_xticklabels()]
        ticklabels = [
            f"\\textsc{{{txt}}}" for txt in ticklabels
        ]
        ax.set_xticklabels(ticklabels, fontsize=16)

        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    elements = [
            Line2D([0], [0], color='C0', marker=markers[0], markersize=8),
            Line2D([0], [0], color='C1', marker=markers[1], markersize=8),
            Line2D([0], [0], color='C2', marker=markers[2], markersize=8),
            Line2D([0], [0], color='C3', marker=markers[3], markersize=8),
            Line2D([0], [0], color='C4', marker=markers[4], markersize=8),
            Line2D([0], [0], color='C5', marker=markers[5], markersize=8)
    ]
    
    fig.legend(
        handles=elements,
        labels=[
            '\\textsc{BaseModel}', '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)',
            '\\textsc{PartFlex} ($\\mu = 2$)', '\\textsc{Fixed}'
        ],
        fontsize=18, loc='lower center', bbox_to_anchor=(0.5, -0.16),
        ncols=3, handlelength=5, frameon=False
    )

    fig.tight_layout()
    fig.savefig('dt_ops.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_dt_ops(d);

In [ ]:
def plot_gm_rm_ops(d):
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,6))
    
    ax = axes.flat[0]

    sns.lineplot(
        data=d, x='GM', y='parcels_outsourced_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_ylabel('Parcels outsourced (\\%)', fontsize=16)
    ax.set_xlabel('\\texttt{GM}', fontsize=16, labelpad=10)
    ax.set_xticks(d.GM.unique())
    for gm in d.GM.unique():
        ax.axvline(x=gm, linestyle='--', color='black', linewidth=0.5, alpha=0.25)

    ax = axes.flat[1]

    sns.lineplot(
        data=d, x='RM', y='parcels_outsourced_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_xlabel('\\texttt{RM}', fontsize=16, labelpad=10)
    ax.set_xticks(d.RM.unique())
    ax.yaxis.tick_right()
    ax.set_ylabel('Parcels outsourced (\\%)', rotation=270, labelpad=20, fontsize=16)
    ax.yaxis.set_label_position('right')
    for rm in d.RM.unique():
        ax.axvline(x=rm, linestyle='--', color='black', linewidth=0.5, alpha=0.25)

    for ax in axes.flat:
        ax.legend().set_visible(False)
        ax.axes.set_axisbelow(True)

        for tl in ax.get_xticklabels():
            tl.set_fontsize(14)
            tl.set_rotation_mode('anchor')
            tl.set_rotation(45)
            tl.set_ha('right')

        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    elements = [
            Line2D([0], [0], color='C0', marker=markers[0], markersize=8),
            Line2D([0], [0], color='C1', marker=markers[1], markersize=8),
            Line2D([0], [0], color='C2', marker=markers[2], markersize=8),
            Line2D([0], [0], color='C3', marker=markers[3], markersize=8),
            Line2D([0], [0], color='C4', marker=markers[4], markersize=8),
            Line2D([0], [0], color='C5', marker=markers[5], markersize=8)
    ]
    
    fig.legend(
        handles=elements,
        labels=[
            '\\textsc{BaseModel}', '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)',
            '\\textsc{PartFlex} ($\\mu = 2$)', '\\textsc{Fixed}'
        ],
        fontsize=14, loc='lower center', bbox_to_anchor=(0.5, -0.12),
        ncols=3, handlelength=5, frameon=False
    )

    fig.tight_layout()
    fig.savefig('gm_rm_ops.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_gm_rm_ops(d);

In [ ]:
def plot_model_oc_ops(d):
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,6))

    colors = dict(zip(d.model_long.sort_values().unique(), sns.color_palette()))
    
    dp = d[d.model_long != 'MBase']
    models = dp.model_long.sort_values().unique()

    ax = axes.flat[0]

    sns.boxplot(
        data=dp, x='model_long', y='courier_moved_pct',
        order=models, hue_order=models, dodge=False,
        hue='model_long', fliersize=0, whis=2,
        palette=colors, ax=ax)
    ax.set_xlabel('')
    ax.set_ylim((0, dp.courier_moved_pct.quantile(0.9999)))
    ax.set_ylabel('Couriers moving between areas (\\%)', fontsize=16)

    for idx, model in enumerate(models):
        avg = dp[dp.model_long == model].courier_moved_pct.median()
        ax.text(x=idx, y=avg, s=f"{avg:.2f}", va='bottom', ha='center', color='white', fontsize=16, fontweight='bold')

    ticklabels = [lbl.get_text() for lbl in ax.get_xticklabels()]
    ticklabels = [models_textsc[lbl] for lbl in ticklabels]
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ticklabels)

    for tl in ax.get_xticklabels():
        tl.set_rotation_mode('anchor')
        tl.set_rotation(45)
        tl.set_ha('right')
        tl.set_fontsize(14)

    ax = axes.flat[1]

    sns.lineplot(
        data=dp, x='OC', y='courier_moved_pct',
        hue='model_long', style='model_long',
        errorbar=None, markers=markers, dashes=False, ax=ax)
    ax.set_xlabel('\\texttt{OC}', fontsize=16, labelpad=10)
    ax.yaxis.tick_right()
    ax.set_ylabel('Couriers moving between areas (\\%)', rotation=270, labelpad=20, fontsize=16)
    ax.yaxis.set_label_position('right')
    ax.legend().set_visible(False)
    for rm in dp.OC.unique():
        ax.axvline(x=rm, linestyle='--', color='black', linewidth=0.5, alpha=0.25)

    for ax in axes.flat:
        ax.axes.set_axisbelow(True)

        for tl in ax.get_xticklabels():
            tl.set_fontsize(14)

        for tl in ax.get_yticklabels():
            tl.set_fontsize(14)

    elements = [
            Line2D([0], [0], color='C1', marker=markers[1], markersize=8),
            Line2D([0], [0], color='C2', marker=markers[2], markersize=8),
            Line2D([0], [0], color='C3', marker=markers[3], markersize=8),
            Line2D([0], [0], color='C4', marker=markers[4], markersize=8),
            Line2D([0], [0], color='C5', marker=markers[5], markersize=8)
    ]
    
    axes.flat[1].legend(
        handles=elements,
        labels=[
            '\\textsc{Flex}', '\\textsc{PartFlex} ($\\mu = 4$)', '\\textsc{PartFlex} ($\\mu = 3$)',
            '\\textsc{PartFlex} ($\\mu = 2$)', '\\textsc{Fixed}'
        ],
        fontsize=14, loc='lower center', bbox_to_anchor=(0.5, -0.5),
        ncols=2, handlelength=5, frameon=False
    )

    fig.tight_layout()
    fig.savefig('model_oc_ops.pdf', dpi=96, bbox_inches='tight')

    return fig, axes

In [ ]:
plot_model_oc_ops(d);

### % outsourced parcels when removing regional and global bounds

In [ ]:
du.groupby('model_long', observed=True).parcels_outsourced_pct.mean()

### Numeric cost comparison

In [ ]:
avg_cost_fixed = d[d.model_long == 'Fixed'].cost_per_parcel.mean()
avg_cost_mbase = d[d.model_long == 'MBase'].cost_per_parcel.mean()
avg_cost_pflex = d[d.model_long == 'PartFlex ($\\mu = 2$)'].cost_per_parcel.mean()

In [ ]:
100 * (avg_cost_fixed / avg_cost_mbase - 1)

In [ ]:
100 * (avg_cost_pflex / avg_cost_mbase - 1)

In [ ]:
med_cost_fixed = d[d.model_long == 'Fixed'].cost_per_parcel.median()
med_cost_mbase = d[d.model_long == 'MBase'].cost_per_parcel.median()
med_cost_pflex = d[d.model_long == 'PartFlex ($\\mu = 2$)'].cost_per_parcel.median()

In [ ]:
100 * (med_cost_fixed / med_cost_mbase - 1)

In [ ]:
100 * (med_cost_pflex / med_cost_mbase - 1)